In [3]:
import time
import logging
import os
import glob
import json
import re
import random
import textwrap
from termcolor import colored
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

import torch
from torch.utils.data import Dataset,DataLoader
import pytorch_lightning as pl

from transformers import T5Tokenizer,T5ForConditionalGeneration,AdamW,get_linear_schedule_with_warmup

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

In [8]:
def extract_data(path):
    with path.open() as f:
        data=json.load(f)
    
    questions=data['data'][0]['paragraphs']
    rows=list()

    for question in questions:
        context=question['context']
        for qa in question['qas']:
            question_=qa['question']
            answers=qa['answers']
            for answer in answers:
                answer_text=answer['text']
                answer_start=answer['answer_start']
                answer_end=answer_start+len(answer_text)

                rows.append({
                    'question':question_,
                    'context':context,
                    'answer_text':answer_text,
                    'answer_start':answer_start,
                    'answer_end':answer_end
                })
    return pd.DataFrame(rows)

In [9]:
extract_data(Path('data/BioASQ/BioASQ-train-factoid-4b.json')).head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [10]:
paths=sorted(list(Path('data/BioASQ').glob("BioASQ-train-*")))
ds=list()
for path in paths:
    ds.append(extract_data(path))
df=pd.concat(ds)
df.head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712
